<a href="https://colab.research.google.com/github/Kavyasree21-hub/Newproj/blob/main/Book_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
books=pd.read_csv('Books.csv')

<ipython-input-5-0b8972a71a1f>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('Books.csv')


In [ ]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [ ]:
books=books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]

In [ ]:
books.rename(columns={'Book-Tile':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher'},inplace=True)

In [ ]:

books.rename(columns={'Book-Title':'title','Year':'year'},inplace=True)

In [ ]:
books.head(2)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [ ]:
users=pd.read_csv('Users.csv')

In [ ]:
users.head(2)
users.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'},inplace=True)

In [ ]:
users.head(2)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [ ]:
ratings=pd.read_csv('Ratings.csv')

In [ ]:
ratings.head(2)
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'rating'},inplace=True)

In [ ]:
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [ ]:
x=ratings['user_id'].value_counts()>200

In [ ]:
y=x[x].index

In [ ]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [ ]:
ratings=ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [ ]:
ratings_with_books=ratings.merge(books,on='ISBN')

In [ ]:
ratings_with_books.shape

(487671, 7)

In [ ]:
number_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
number_rating.head(2)
number_rating.rename(columns={'rating':'no.of rating'},inplace=True)

In [ ]:
number_rating.head(2)

,title,no.of rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [ ]:
finall_rating=ratings_with_books.merge(number_rating,on='title');

In [ ]:
finall_rating.shape

(487671, 8)

In [ ]:
finall_rating=finall_rating[finall_rating['no.of rating']>=50]

In [ ]:
finall_rating.drop_duplicates(['user_id','title'],inplace=True)

In [ ]:
finall_rating.shape

(59850, 8)

In [ ]:
book_pivot=finall_rating.pivot_table(columns='user_id',index='title',values='rating')

In [ ]:
book_pivot.fillna(0,inplace=True)

In [ ]:
book_pivot.shape

(742, 888)

In [ ]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(book_pivot)

In [ ]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [ ]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

In [ ]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
distances,suggestions=model.kneighbors(book_pivot.iloc[54,:].values.reshape(1,-1),n_neighbors=6)

In [ ]:
suggestions

array([[ 54, 184, 291, 440, 393, 372]])

In [ ]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['Animal Farm', 'Exclusive', 'Jacob Have I Loved', 'Second Nature',
       'Pleading Guilty', 'No Safe Place'],
      dtype='object', name='title')


In [ ]:
np.where(book_pivot.index=='Animal Farm')[0][0]

54

In [ ]:
def recommend_book(book_name):
  book_id=np.where(book_pivot.index==book_name)[0][0]
  distances,suggestions=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
  for i in range(len(suggestions)):
    if i==0:
      print("The suggestions for",book_name,"are :")
    if not i:
      print(book_pivot.index[suggestions[i]])

In [ ]:
recommend_book('Animal Farm')

The suggestions for Animal Farm are :
Index(['Animal Farm', 'Exclusive', 'Jacob Have I Loved', 'Second Nature',
       'Pleading Guilty', 'No Safe Place'],
      dtype='object', name='title')
